In [2]:
%pip install ipywidgets -q
%pip install prophet -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from prophet import Prophet

In [4]:
cash_flow_components = pd.read_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv", header=[0, 1], index_col=0)
cash_flow_components

CSAN3.SA                      \
                    2023-06-30 00:00:00 2023-03-31 00:00:00   
End Cash Position          1.573929e+10        1.138543e+10   
Changes In Cash            4.433235e+09       -1.896770e+09   
Financing Cash Flow        2.695912e+09       -2.301659e+09   

                                                             \
                    2022-12-31 00:00:00 2022-09-30 00:00:00   
End Cash Position          1.330172e+10        1.548710e+10   
Changes In Cash           -2.175234e+09        2.907373e+09   
Financing Cash Flow        5.534252e+09        5.106019e+09   

                               CIEL3.SA                      \
                    2023-06-30 00:00:00 2023-03-31 00:00:00   
End Cash Position          9.518960e+08        2.351092e+09   
Changes In Cash           -1.399195e+09        4.682260e+08   
Financing Cash Flow        3.320040e+08       -6.628960e+08   

                                                             \
                    2022-12-31 00:00:00 2022-09-30 00:00:00   
End Cash Position          1.882866e+09        1.014558e+09   
Changes In Cash            8.683080e+08       -7.416660e+08   
Financing Cash Flow       -5.685720e+08       -7.827640e+08   

                               EZTC3.SA                      ...  \
                    2023-06-30 00:00:00 2023-03-31 00:00:00  ...   
End Cash Position            42473000.0          33602000.0  ...   
Changes In Cash               1656000.0          -8287000.0  ...   
Financing Cash Flow          69204000.0          38677000.0  ...   

                               ALPA4.SA                      \
                    2022-12-31 00:00:00 2022-09-30 00:00:00   
End Cash Position           647517000.0         240984000.0   
Changes In Cash             407010000.0         -30181000.0   
Financing Cash Flow         804951000.0         157054000.0   

                               B3SA3.SA                      \
                    2023-06-30 00:00:00 2023-03-31 00:00:00   
End Cash Position          3.525740e+08        3.245230e+08   
Changes In Cash            3.154100e+07       -3.635900e+07   
Financing Cash Flow       -1.453471e+09       -1.309368e+09   

                                                             \
                    2022-12-31 00:00:00 2022-09-30 00:00:00   
End Cash Position          3.622420e+08        3.184870e+08   
Changes In Cash            4.375500e+07        3.698500e+07   
Financing Cash Flow       -1.694528e+09       -2.396088e+09   

                               KEPL3.SA                      \
                    2023-06-30 00:00:00 2023-03-31 00:00:00   
End Cash Position           251482000.0         199128000.0   
Changes In Cash              52354000.0         -55326000.0   
Financing Cash Flow        -104949000.0           -995000.0   

                                                             
                    2022-12-31 00:00:00 2022-09-30 00:00:00  
End Cash Position           254454000.0         200402000.0  
Changes In Cash              54052000.0          -8143000.0  
Financing Cash Flow         -51757000.0         -51690000.0  

[3 rows x 432 columns]

## Pré-processamento dos dados

In [5]:
from utils import get_columns_from_header_0

tickers = get_columns_from_header_0(cash_flow_components)
tickers

['MGLU3.SA',
 'ASAI3.SA',
 'SANB11.SA',
 'YDUQ3.SA',
 'PSSA3.SA',
 'CIEL3.SA',
 'CMIN3.SA',
 'VBBR3.SA',
 'AURE3.SA',
 'TIMS3.SA',
 'BRFS3.SA',
 'SMTO3.SA',
 'USIM5.SA',
 'GGBR4.SA',
 'FESA4.SA',
 'PETR4.SA',
 'ABEV3.SA',
 'RAIL3.SA',
 'CPFE3.SA',
 'RAPT4.SA',
 'GOLL4.SA',
 'DXCO3.SA',
 'TASA4.SA',
 'CCRO3.SA',
 'ITSA4.SA',
 'RRRP3.SA',
 'KEPL3.SA',
 'ENEV3.SA',
 'UNIP6.SA',
 'SUZB3.SA',
 'TRIS3.SA',
 'LEVE3.SA',
 'CVCB3.SA',
 'LAVV3.SA',
 'JBSS3.SA',
 'VALE3.SA',
 'EGIE3.SA',
 'PETR3.SA',
 'AGRO3.SA',
 'EZTC3.SA',
 'ELET3.SA',
 'GOAU4.SA',
 'SOMA3.SA',
 'FLRY3.SA',
 'MRVE3.SA',
 'CPLE6.SA',
 'SBSP3.SA',
 'IRBR3.SA',
 'BHIA3.SA',
 'TRPL4.SA',
 'WEGE3.SA',
 'EMBR3.SA',
 'NTCO3.SA',
 'JHSF3.SA',
 'CSNA3.SA',
 'WIZC3.SA',
 'KLBN11.SA',
 'BRSR6.SA',
 'CYRE3.SA',
 'DIRR3.SA',
 'ROMI3.SA',
 'HYPE3.SA',
 'IGTI11.SA',
 'ABCB4.SA',
 'PRIO3.SA',
 'TGMA3.SA',
 'CRFB3.SA',
 'BEEF3.SA',
 'PETZ3.SA',
 'ENGI11.SA',
 'RANI3.SA',
 'CMIG3.SA',
 'SLCE3.SA',
 'ALSO3.SA',
 'RENT3.SA',
 'BPAC11.SA',
 'ELET6

In [6]:
from typing import List
from pandas import DataFrame


def to_time_series(ticker_df: DataFrame) -> List[DataFrame]:
  time_series = []

  for column in ticker_df.columns:
    column_df = DataFrame(ticker_df[column])
    column_df.insert(loc=0, column="ds", value=pd.to_datetime(column_df.index))
    column_df.rename(columns={column: "y"}, inplace=True)
    column_df.reset_index(drop=True, inplace=True)
    column_df = column_df.sort_values(by="ds")

    time_series.append(column_df)

  return time_series


## Previsão dos componentes do fluxo de caixa

In [7]:
def predict(ticker_df: DataFrame, n_quarters: int = 1) -> DataFrame:
    ticker_time_series = to_time_series(ticker_df)

    predictions = pd.DataFrame()

    for i, dataset in enumerate(ticker_time_series):
        if len(dataset) < 2:
            continue

        model = Prophet()
        model.fit(dataset)

        # Prever 1 ano de um dos componentes do fluxo de caixa
        future = model.make_future_dataframe(periods=n_quarters, freq="BQ")
        forecast = model.predict(future)

        forecast = forecast[["ds", "yhat"]].rename(
            columns={"yhat": ticker_df.columns[i]})
        forecast.sort_values(by="ds")
        forecast.set_index("ds", inplace=True)
        forecast.index.name = None
        forecast = forecast.transpose()

        if predictions.empty:
            predictions = forecast
            continue

        predictions = pd.concat([predictions, forecast])

    return predictions

In [11]:
def create_predicted_cash_flow_components(n_quarters: int):
    predicted_cash_flow_components = DataFrame()

    for ticker in tickers:
        ticker_df = cash_flow_components[ticker].transpose()
        predictions = predict(ticker_df, n_quarters)

        predictions.columns = pd.MultiIndex.from_product(
            [[ticker], predictions.columns])

        if predicted_cash_flow_components.empty:
            predicted_cash_flow_components = predictions
            continue

        predicted_cash_flow_components = pd.concat(
            [predicted_cash_flow_components, predictions], axis=1)
        
    predicted_cash_flow_components.to_csv("./assets/predicted_cash_flow_components.csv")

    return predicted_cash_flow_components

In [12]:
create_predicted_cash_flow_components(4 * 2)

18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:31 - cmdstanpy - INFO - Chain [1] done processing
18:00:31 - cmdstanpy - INFO - Chain [1] start processing
18:00:32 - cmdstanpy - INFO - Chain [1] done processing
18:00:32 - cmdstanpy - INFO - Chain [1] start processing
18:00:32 - cmdstanpy - INFO - Chain [1] done processing
18:00:32 - cmdstanpy - INFO - Chain [1] start processing
18:00:32 - cmdstanpy - INFO - Chain [1]

MGLU3.SA                                            \
                       2022-09-30    2022-12-31    2023-03-31    2023-06-30   
End Cash Position    1.992972e+09  2.017534e+09  2.041562e+09  2.065857e+09   
Changes In Cash      1.819585e+08  1.247799e+08  6.884424e+07  1.228711e+07   
Financing Cash Flow -7.865243e+08 -7.941961e+08 -8.017012e+08 -8.092895e+08   

                                                                             \
                       2023-09-29    2023-12-29    2024-03-29    2024-06-28   
End Cash Position    2.090151e+09  2.114446e+09  2.138741e+09  2.163036e+09   
Changes In Cash     -4.427002e+07 -1.008272e+08 -1.573843e+08 -2.139414e+08   
Financing Cash Flow -8.168779e+08 -8.244663e+08 -8.320547e+08 -8.396431e+08   

                                                 ...      ALPA4.SA  \
                       2024-09-30    2024-12-31  ...    2023-03-31   
End Cash Position    2.188131e+09  2.212693e+09  ...  4.633355e+08   
Changes In Cash     -2.723631e+08 -3.295417e+08  ...  3.419020e+06   
Financing Cash Flow -8.474816e+08 -8.551534e+08  ...  1.964306e+08   

                                                                             \
                       2023-06-30    2023-09-29    2023-12-29    2024-03-29   
End Cash Position    4.997102e+08  5.360849e+08  5.724596e+08  6.088343e+08   
Changes In Cash     -6.131850e+07 -1.260560e+08 -1.907935e+08 -2.555310e+08   
Financing Cash Flow  8.066334e+07 -3.510391e+07 -1.508712e+08 -2.666384e+08   

                                                                             \
                       2024-06-28    2024-09-30    2024-12-31    2025-03-31   
End Cash Position    6.452090e+08  6.827829e+08  7.195573e+08  7.555323e+08   
Changes In Cash     -3.202686e+08 -3.871403e+08 -4.525892e+08 -5.166153e+08   
Financing Cash Flow -3.824057e+08 -5.019894e+08 -6.190288e+08 -7.335239e+08   

                                   
                       2025-06-30  
End Cash Position    7.919070e+08  
Changes In Cash     -5.813528e+08  
Financing Cash Flow -8.492911e+08  

[3 rows x 1311 columns]